In [1]:
import path_configs # noqa
import tensorflow as tf
import settings
from datetime import datetime
import os
settings.init()

from modules.ClassifierGenerators import (TrainClassifierGenerator,  # noqa
                                          ValidationClassifierGenerator)  # noqa

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


# assumes that array is not zero
def scaled(tensor):
    return (tensor-tf.math.reduce_min(tensor))/(tf.math.reduce_max(tensor)-tf.
                                                math.reduce_min(tensor))

TRAIN_DATA_LEN: 3720334
VAL_DATA_LEN: 1593071


In [3]:
for x in TrainClassifierGenerator():
    print(x[0].shape)

(256, 64, 1025)
(256, 64, 1025)
(256, 64, 1025)
(256, 64, 1025)
(256, 64, 1025)
(256, 64, 1025)
(256, 64, 1025)
(256, 64, 1025)
(256, 64, 1025)
(256, 64, 1025)
(256, 64, 1025)
(256, 64, 1025)
(256, 64, 1025)
(256, 64, 1025)
(256, 64, 1025)
(256, 64, 1025)


KeyboardInterrupt: 

In [4]:
def base_model(encoder_output):
    base_layer = tf.keras.layers.RepeatVector(settings.AE_TIMESTEPS)(encoder_output)
    base_layer = tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                                        activation='relu')(base_layer)
    base_layer = tf.keras.layers.Dropout(0.4)(base_layer)
    base_layer = tf.keras.layers.Conv1D(filters=128, kernel_size=3,
                                        activation='relu')(base_layer)
    base_layer = tf.keras.layers.Dropout(0.4)(base_layer)
    base_layer = tf.keras.layers.Flatten()(base_layer)
    return base_layer


# age branch
def build_age_branch(inputs):
    age_layer = tf.keras.layers.Dense(512, activation='relu')(inputs)
    age_layer = tf.keras.layers.Dropout(0.4)(age_layer)
    age_layer = tf.keras.layers.Dense(256, activation='relu')(age_layer)
    age_layer = tf.keras.layers.Dropout(0.4)(age_layer)
    age_output = tf.keras.layers.Dense(9, activation='sigmoid',
                                       name='age')(age_layer)
    return age_output


# gender branch,
def build_gender_branch(inputs):
    gender_layer = tf.keras.layers.Dense(512, activation='relu')(inputs)
    gender_layer = tf.keras.layers.Dropout(0.4)(gender_layer)
    gender_layer = tf.keras.layers.Dense(256, activation='relu')(gender_layer)
    gender_layer = tf.keras.layers.Dropout(0.4)(gender_layer)
    gender_output = tf.keras.layers.Dense(1, activation='sigmoid',
                                          name='gender')(gender_layer)
    return gender_output


def assemble_full_model():
    autoencoder = tf.keras.models.load_model('model_autoencoder_final.h5')
    encoder_layer = autoencoder.layers[1]
    encoder_layer = tf.keras.models.Model(encoder_layer.layers[0].input,
                                            encoder_layer.layers[2].output)
    encoder_layer.trainable = False
    encoder_input = encoder_layer.input
    inputs = base_model(encoder_layer.output)

    age_branch = build_age_branch(inputs)
    gender_branch = build_gender_branch(inputs)

    model = tf.keras.models.Model(inputs=encoder_input,
                                  outputs=[age_branch, gender_branch],
                                  name="voice_net")
    return model

model = assemble_full_model()

In [5]:
model.compile(
    optimizer='adam',
    loss={'age': 'sparse_categorical_crossentropy',
          'gender': 'categorical_crossentropy'},
    loss_weights={'age': 1,
                  'gender': 1},
    metrics=['accuracy'])
print(model.summary())

now = datetime.now()
date_time = now.strftime("%m_%d_%Y_%H_%M")

NAME = ("main_model_" + date_time)

for folder_name in ['logs', 'saved_models', os.path.join('saved_models',
                                                         'checkpoints')]:
    try:
        os.mkdir(folder_name)
        print("Directory", folder_name,  "created ")
    except FileExistsError:
        print("Directory", folder_name,  "already exists")

Model: "voice_net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 1025)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 64, 256)      1312768     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 128)          197120      lstm[0][0]                       
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 64, 128)      0           lstm_1[0][0]                     
__________________________________________________________________________________________

In [6]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=os.path.join('logs', str(NAME))),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join('saved_models', 'checkpoints',
                              '{}.h5'.format(NAME)),
        monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
    ]

tf.keras.utils.plot_model(model,
                          to_file=os.path.join('figures',
                                               'main_model_schema.pdf'),
                          show_shapes=True, expand_nested=True)

model.fit(
    TrainClassifierGenerator().prefetch(tf.data.experimental.AUTOTUNE),
    epochs=settings.MODEL_EPOCHS,
    steps_per_epoch=(settings.TRAIN_DATA_LEN//2)//settings.MODEL_BATCH_SIZE,
    verbose=1,
    validation_data=ValidationClassifierGenerator()
    .prefetch(tf.data.experimental.AUTOTUNE),
    validation_steps=(settings.VAL_DATA_LEN//2)//settings.MODEL_BATCH_SIZE,
    callbacks=callbacks
    )

model.save(os.path.join('saved_models', '{}.h5'.format(NAME)))

Epoch 1/6
   1/7266 [..............................] - ETA: 1s - loss: 1.4730 - age_loss: 1.4730 - gender_loss: 1.6112e-07 - age_accuracy: 0.0664 - gender_accuracy: 0.0469WARNING:tensorflow:From /home/stshalson/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
  74/7266 [..............................] - ETA: 24:43 - loss: 0.9378 - age_loss: 0.9378 - gender_loss: 1.9317e-07 - age_accuracy: 0.1065 - gender_accuracy: 0.1557

KeyboardInterrupt: 